In [2]:
import requests
import numpy as np
import pandas as pd

In [3]:
# Resource ids are obtained from the data.gov.sg website (Data API)
resource_ids = {
    "RH" : '4631174f-9858-463d-8a88-f3cb21588c67',
    "Rainfall_count" : '8b94f596-91fd-4545-bf9e-7a426493b674',
    "Rainfall_total" : '778814b8-1b96-404b-9ac9-68d6c00e637b',
    "SA Temp" : '07654ce7-f97f-49c9-81c6-bd41beba4e96',
    "Sunshine" : '0230819f-1c83-4980-b738-56136d6dc300'
}

# since we want all the data, we set the limit to be the maximum value of a 64-bit integer
limit = 9223372036854775807

data = []
# Meteorological data
for key in resource_ids:
    url = f"https://data.gov.sg/api/action/datastore_search?resource_id={resource_ids[key]}&limit={limit}"
    r = requests.get(url)
    data.append(pd.DataFrame(r.json().get('result').get('records')))

In [4]:
# Merge all the data in the list into one dataframe
df = pd.concat(data, join='inner', axis=1)

# Get all the rows for first column of 'month'
date_value = df['month'].iloc[:,0]

# Make the date a new column at the start (with the name called 'date')
df.insert(loc=0, column='date', value=date_value)

# Delete the duplicate columns ('month' and '_id')
df.drop(['month', '_id'], axis=1, inplace=True)
display(df)

,date,mean_rh,no_of_rainy_days,total_rainfall,mean_temp,mean_sunshine_hrs
0,1982-01,81.2,10,107.1,25.9,5.6
1,1982-02,79.5,5,27.8,27.1,7.6
2,1982-03,82.3,11,160.8,27.2,5.1
3,1982-04,85.9,14,157,27,4.7
4,1982-05,83.2,10,102.2,28,5.8
...,...,...,...,...,...,...
487,2022-08,77.4,15,141.4,28.1,5.5
488,2022-09,76.9,14,121,27.9,3.8
489,2022-10,78.7,27,279,27.6,3.5
490,2022-11,79.9,26,313.8,27.4,3.8


In [5]:
# Check for missing values
df.isnull().sum()

date                 0
mean_rh              0
no_of_rainy_days     0
total_rainfall       0
mean_temp            0
mean_sunshine_hrs    0
dtype: int64

In [6]:
# Reformat the date column into date object
df['date'] = pd.to_datetime(df['date'])

# Convert all data (except date) to numeric
df.iloc[:,1:] = df.iloc[:,1:].apply(pd.to_numeric)

In [7]:
# Resample the data and reduce its granularity to yearly (so that we can compare with our other data)
yearly_df = df.resample('Y', on='date').aggregate({'mean_rh': ['mean', 'median', 'max', 'min'], 'no_of_rainy_days': 'sum', 'total_rainfall': 'sum', 'mean_temp': ['mean', 'median', 'max', 'min'], 'mean_sunshine_hrs': ['mean', 'median', 'max', 'min']})

# Rename the columns based on their aggregation functions
yearly_df.columns = ['mean_annual_rh', 'median_annual_rh', 'max_annual_rh', 'min_annual_rh', 'annual_num_of_rainy_days', 'total_annual_precipitation', 'mean_annual_temp', 'median_annual_temp', 'max_annual_temp', 'min_annual_temp', 'mean_annual_sunshine_hrs', 'median_annual_sunshine_hrs', 'max_annual_sunshine_hrs', 'min_annual_sunshine_hrs']

# Reset the index so that the date is a column again
yearly_df.reset_index(inplace=True)

# Convert date column to only have year
yearly_df['date'] = yearly_df['date'].dt.year

# Add a new column total_mean_precipitation which is total_annual_precipitation / num of days in the year
# A leap year is exactly divisible by 4 except for century years (years ending with 00). The century year is a leap year only if it is perfectly divisible by 400.
num_of_days = yearly_df['date'].apply(lambda x: 366 if ((x % 4 == 0 and x % 100 != 0) or (x % 400 == 0  and x % 100 == 0)) else 365)
yearly_df.insert(loc=7, column='total_mean_precipitation', value=(yearly_df['total_annual_precipitation'])/num_of_days)

# Round the data to 2 decimal places
yearly_df = yearly_df.round(2)
display(yearly_df)

,date,mean_annual_rh,median_annual_rh,max_annual_rh,min_annual_rh,annual_num_of_rainy_days,total_annual_precipitation,total_mean_precipitation,mean_annual_temp,median_annual_temp,max_annual_temp,min_annual_temp,mean_annual_sunshine_hrs,median_annual_sunshine_hrs,max_annual_sunshine_hrs,min_annual_sunshine_hrs
0,1982,81.97,81.35,87.7,77.6,130,1581.7,4.33,27.36,27.35,28.4,25.9,5.75,5.60,7.6,3.3
1,1983,82.28,82.85,88.0,76.0,145,1993.7,5.46,27.67,27.85,29.4,25.4,5.62,5.65,9.3,2.7
2,1984,84.62,84.10,86.7,82.5,191,2686.7,7.34,26.77,27.00,27.5,25.5,5.08,5.30,6.4,3.3
3,1985,83.21,83.60,86.2,79.4,181,1483.9,4.07,27.12,27.00,28.2,26.1,5.59,5.55,8.7,4.1
4,1986,82.87,84.00,86.6,77.3,174,2536.1,6.95,27.10,26.90,28.3,25.7,5.52,5.50,7.2,3.9
5,1987,82.87,83.00,86.0,79.0,151,2102.8,5.76,27.38,27.55,28.7,25.5,5.92,5.70,9.2,4.0
6,1988,83.84,84.15,86.8,81.1,173,2598.6,7.10,27.23,27.20,28.4,26.2,5.38,5.50,6.7,3.7
7,1989,84.35,84.25,87.6,78.3,173,2463.2,6.75,26.92,26.95,27.7,26.3,5.53,5.50,6.7,4.5
8,1990,82.21,82.50,86.0,77.6,142,1523.8,4.17,27.59,27.55,28.6,26.2,6.20,5.90,8.9,4.6
9,1991,83.13,82.30,90.7,78.7,146,1877.0,5.14,27.43,27.60,28.6,25.6,5.38,5.45,7.1,3.0


In [9]:
# Write data to csv file
yearly_df.to_csv('./datasrc/meteorological.csv', index=False)